<a href="https://colab.research.google.com/github/justinpatel/Play-Generator-ML/blob/master/Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

Get the text file

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In order to get your own data from local machine

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print("Length: {}".format(len(text)))

Length: 1115394


In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


preprocess of data

In [4]:
vocab = sorted(set(text))

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

In [5]:
text_as_int = text_to_int(text)
type(text_as_int)

numpy.ndarray

In [6]:
def int_to_text(intt):
  try:
    intt = intt.numpy()
  except:
    pass
  return ''.join(idx2char[intt])

In [7]:
print(int_to_text(text_as_int[:10]))

First Citi


creating training example

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  output_text = chunk[1:]
  return input_text, output_text

In [10]:
dataset = sequences.map(split_input_target)

In [11]:
for x, y in dataset.take(1):
  print(x)
  print(y)

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)
tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int64)


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

In [13]:
data = dataset.shuffle(10000).batch(64, drop_remainder=True)

In [17]:
data

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, recurrent_initializer="glorot_uniform", return_sequences=True, stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

In [18]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


creating a loss function

In [20]:
for input, target in data.take(1):
  input_prediction = model(input)
  print(input_prediction.shape)

(64, 100, 65)


In [29]:
input_prediction[0][0]

<tf.Tensor: shape=(65,), dtype=float32, numpy=
array([ 1.6117176e-02, -2.0026788e-03,  3.1119646e-03,  3.1135147e-03,
       -1.2254943e-02,  9.2881499e-05,  1.0527703e-02,  7.6015894e-03,
       -2.9979768e-05,  5.0755949e-03, -3.7412741e-03, -1.1748727e-02,
        2.4860706e-03,  8.8417623e-03, -1.0405013e-02, -5.4631545e-03,
       -3.1382244e-03, -3.0963931e-03, -1.0823305e-02, -2.9921648e-03,
        2.1320288e-03,  7.3236087e-03,  7.0907059e-04,  3.7204255e-03,
       -5.2960813e-03, -3.5010525e-03, -8.8643972e-03, -9.3327798e-03,
       -4.7009005e-03,  4.8857979e-03,  5.0896346e-03, -7.1866754e-03,
        6.6136464e-04, -7.5377487e-03, -1.0656528e-02,  1.3267454e-03,
       -3.6001953e-03,  6.8235835e-03,  4.9132318e-04, -1.3757628e-02,
       -6.8012616e-03, -1.5089649e-03,  7.3901657e-04, -3.4102297e-03,
       -1.5760972e-03,  5.9014275e-03, -7.0760027e-04,  5.2388581e-03,
        2.8587179e-04,  7.7912176e-04, -6.8626525e-03, -1.4999339e-03,
        4.2021088e-03, -1.5254

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer="adam", loss=loss)

configure checkpoints

In [22]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 82ms/step - loss: 2.7704
Epoch 2/50
172/172 [==============================] - 14s 82ms/step - loss: 2.0763
Epoch 3/50
172/172 [==============================] - 14s 79ms/step - loss: 1.8158
Epoch 4/50
172/172 [==============================] - 13s 77ms/step - loss: 1.6591
Epoch 5/50
172/172 [==============================] - 13s 76ms/step - loss: 1.5592
Epoch 6/50
172/172 [==============================] - 13s 77ms/step - loss: 1.4912
Epoch 7/50
172/172 [==============================] - 14s 79ms/step - loss: 1.4410
Epoch 8/50
172/172 [==============================] - 14s 81ms/step - loss: 1.4027
Epoch 9/50
172/172 [==============================] - 14s 79ms/step - loss: 1.3707
Epoch 10/50
172/172 [==============================] - 14s 79ms/step - loss: 1.3447
Epoch 11/50
172/172 [==============================] - 13s 78ms/step - loss: 1.3218
Epoch 12/50
172/172 [==============================] - 13s 78ms/step - loss: 1.2997
E

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM,RNN_UNITS,batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [27]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("type something: ")
print(generate_text(model, inp))

type something: romeo
romeo's and no judgment patience.

KING EDWARD IV:
Alas, poor Clarence! Should a husband be made;
It is a pursuirrop for our happy horse.
Thank you, sir:
That my tears are thence for he is:
Behold, wise man, would give you all to Rivest hear discold.

GLOUCESTER:
He hath eaten,
Should knave him, we are shed faults on Lancaster.

Provost:
Has that my music be my helmet and Elposter, Edward wepl'd and kill thy
soldiers; and their power is dead like the forfounce man's
execution, and then reconciled flight,
And nobly dead still it be: so wise as it,
Shambly entertain'st thy words to see the end of the back;
And in defend me week and soldiers;
Which else would pursue the miraccouses; thou kill'st men!

PROSPERO:
Hast there, my lord,
Embarquestick; throw you that ease.

LORD ROSS:
It stands too farl!
Thou wrong'st thee roar'd to question, there he were undone!
If I should wear peace under your forward wish
you speak. A crey-lower wish the sun.
How long is't since?

MEN